# 黑色金属 - 螺纹钢

In [1]:
# 设置工作目录
import os
os.getcwd()
if os.name == 'posix':
    os.chdir('/Volumes/Repository/Projects/ffa/')
else:
    os.chdir("E:\\Document\\Project\\ffa")

In [9]:
# 加载依赖模块
import pandas as pd
import numpy as np
import akshare as ak
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, date
import importlib
import commodity
import json
from datetime import timedelta
import dataworks as dw
    

In [14]:
# 重新加载salary计算模块
importlib.reload(commodity)
importlib.reload(dw)

<module 'dataworks' from '/Volumes/Repository/Projects/ffa/dataworks.py'>

## 数据准备与预处理

In [11]:
symbol_id = 'RB'
symbol_name = '螺纹钢'
fBasePath = 'steel/data/mid-stream/螺纹钢/'
json_file = './steel/setting.json'

### 数据索引设置

In [12]:
# 品种数据索引初始化
# 首次使用json配置文件存取品种的数据索引
data_index = {'主力合约': {'Name': "dominant_contract", 'Source':'SQLite', 'Path': 'basis', 'Field':'dominant_contract', 'DataFrame': "df_basis"},
                '近月合约': {'Name': "near_contract", 'Source':'SQLite', 'Path': 'basis', 'Field':'near_contract', 'DataFrame': "df_basis"},
                # '主力合约收盘价': {'Name': "dominant_close_price", 'Source':'Choice', 'Path': fBasePath + '螺纹钢期货价格.xlsx', 'Field':'期货收盘价（主力）：螺纹钢', 'DataFrame': "df_dominant"},
                # '主力合约结算价': {'Name': "dominant_settle_price", 'Source':'Choice', 'Path': fBasePath + '螺纹钢期货价格.xlsx', 'Field':'期货结算价（主力）：螺纹钢', 'DataFrame': "df_dominant"},
                '主力合约收盘价': {'Name': "dominant_close_price", 'Source':'SQLite', 'Path': 'dominant', 'Field':'收盘价', 'DataFrame': "df_dominant"},
                '主力合约结算价': {'Name': "dominant_settle_price", 'Source':'SQLite', 'Path': 'dominant', 'Field':'动态结算价', 'DataFrame': "df_dominant"},                
                '现货价格': {'Name': "spot_price", 'Source':'SQLite', 'Path': 'basis', 'Field':'spot_price', 'DataFrame': "df_basis"},
                '基差': {'Name': "basis", 'Source':'SQLite', 'Path':'‘basis', 'Field':'dom_basis', 'DataFrame': "df_basis"},
                '基差率': {'Name': "basis_rate", 'Source':'SQLite', 'Path': 'basis', 'Field':'dom_basis_rate', 'DataFrame': "df_basis"},
                '产量': {'Name': "production", 'Source':'Choice', 'Path': fBasePath + '螺纹钢产量.xlsx', 'Field':'产量：钢筋：全国：当月值', 'DataFrame': "df_prodoction"},
                '销量': {'Name': "sales", 'Source':'Choice', 'Path': fBasePath + '螺纹钢销量.xlsx', 'Field':'销量：钢筋：累计值', 'DataFrame': "df_sales"},
                '库存': {'Name': "instock", 'Source':'Choice', 'Path': fBasePath + '螺纹钢库存.xlsx', 'Field':'库存：螺纹钢：合计', 'DataFrame': "df_instock"},
                '仓单': {'Name': "receipt", 'Source':'Choice', 'Path': fBasePath + '螺纹钢库存.xlsx', 'Field':'仓单数量：螺纹钢', 'DataFrame': "df_instock"}
                }
profit_formula = {'Name': 'profit_formula', 'Factor': {'铁矿石': 1.6, '焦炭': 0.6}, '其他成本': 1200}
dominant_months = [1, 5, 10]
exchange_id = 'shfe'
symbol_setting = {'DataIndex': data_index,
                  'ProfitFormula': profit_formula,
                  'DominantMonths': dominant_months,
                  'ExchangeID': exchange_id}


In [15]:
# 构造品种数据访问对象
dws = dw.DataWorks()
symbol = commodity.SymbolData(symbol_id, symbol_name, json_file, symbol_setting)
# symbol = commodity.SymbolData(symbol_id, symbol_name, json_file)
symbol.merge_data(dws)
symbol.get_spot_months()
# symbol_j = commodity.SymbolData('J', '焦炭', json_file)
# symbol_j.merge_data()
# symbol_i = commodity.SymbolData('I', '铁矿石', json_file)
# symbol_i.merge_data()

In [5]:
exchange_id = symbol.symbol_setting['ExchangeID']
df_contract_data = pd.read_excel('data/' + exchange_id + '.xlsx')

In [6]:
df_symbol_contract = df_contract_data[df_contract_data['variety']=='RB'].copy()
df_symbol_contract['date'] = pd.to_datetime(df_symbol_contract['date'].astype(str), format='%Y%m%d')
current_date = datetime.strptime('20110104', '%Y%m%d')
df_symbol_contract_date = df_symbol_contract[df_symbol_contract['date']==current_date]
df_symbol_contract_date

,symbol,date,open,high,low,close,volume,open_interest,turnover,settle,pre_settle,variety
48,RB1101,2011-01-04,4929.0,4939.0,4920.0,4939.0,1924,4380.0,9.491092e+06,4933.0,4901.0,RB
49,RB1102,2011-01-04,4864.0,4892.0,4864.0,4864.0,112,1262.0,5.449920e+05,4866.0,4863.0,RB
50,RB1103,2011-01-04,4830.0,4830.0,4818.0,4819.0,208,3732.0,1.002560e+06,4820.0,4821.0,RB
51,RB1104,2011-01-04,4800.0,4828.0,4800.0,4821.0,238,6578.0,1.146684e+06,4818.0,4804.0,RB
52,RB1105,2011-01-04,4830.0,4843.0,4805.0,4822.0,275428,576934.0,1.328389e+09,4823.0,4803.0,RB
53,RB1106,2011-01-04,4834.0,4834.0,4805.0,4823.0,280,1366.0,1.349600e+06,4820.0,4795.0,RB
54,RB1107,2011-01-04,4839.0,4839.0,4805.0,4824.0,54,5036.0,2.600640e+05,4816.0,4786.0,RB
55,RB1108,2011-01-04,4803.0,4840.0,4803.0,4836.0,18,346.0,8.686800e+04,4826.0,4803.0,RB
56,RB1109,2011-01-04,4850.0,4861.0,4823.0,4861.0,560,2172.0,2.716000e+06,4850.0,4823.0,RB
57,RB1110,2011-01-04,4900.0,4926.0,4897.0,4918.0,30656,71848.0,1.507049e+08,4916.0,4877.0,RB


In [5]:
import dataworks as dw

In [18]:
importlib.reload(dw)

<module 'dataworks' from 'E:\\Document\\Project\\ffa\\dataworks.py'>

In [9]:
d = dw.DataWorks()
exchange_id = symbol.symbol_setting['ExchangeID']
df_term = d.get_data_by_symbol(exchange_id, ['symbol', 'date', 'close', 'volume', 'settle', 'variety'], symbol.id)
df_term

,symbol,date,close,volume,settle,variety
0,RB1101,2011-01-04,4939.0,1924,4933.0,RB
1,RB1102,2011-01-04,4864.0,112,4866.0,RB
2,RB1103,2011-01-04,4819.0,208,4820.0,RB
3,RB1104,2011-01-04,4821.0,238,4818.0,RB
4,RB1105,2011-01-04,4822.0,275428,4823.0,RB
...,...,...,...,...,...,...
37648,RB2407,2023-11-30,3954.0,12755,3946.0,RB
37649,RB2408,2023-11-30,3951.0,997,3947.0,RB
37650,RB2409,2023-11-30,3942.0,2286,3926.0,RB
37651,RB2410,2023-11-30,3910.0,8761,3897.0,RB


In [8]:

current_date = datetime.strptime('20110104', '%Y%m%d')
df= d.get_data_by_symbol('shfe', ['symbol', 'date', 'close', 'volume', 'settle', 'variety'], 'RB')
df[df['date']==current_date]

,symbol,date,close,volume,settle,variety
0,RB1101,2011-01-04,4939.0,1924,4933.0,RB
1,RB1102,2011-01-04,4864.0,112,4866.0,RB
2,RB1103,2011-01-04,4819.0,208,4820.0,RB
3,RB1104,2011-01-04,4821.0,238,4818.0,RB
4,RB1105,2011-01-04,4822.0,275428,4823.0,RB
5,RB1106,2011-01-04,4823.0,280,4820.0,RB
6,RB1107,2011-01-04,4824.0,54,4816.0,RB
7,RB1108,2011-01-04,4836.0,18,4826.0,RB
8,RB1109,2011-01-04,4861.0,560,4850.0,RB
9,RB1110,2011-01-04,4918.0,30656,4916.0,RB


In [22]:
df['date'].idxmax()

37641

In [7]:
df_symbol_contract_date['symbol'].unique()

array(['RB1101', 'RB1102', 'RB1103', 'RB1104', 'RB1105', 'RB1106',
       'RB1107', 'RB1108', 'RB1109', 'RB1110', 'RB1111', 'RB1112'],
      dtype=object)

In [8]:
import sqlite3
conn = sqlite3.connect('data/futures.db')
df_contract_data.to_sql('shfe', conn, if_exists='replace', index=False)
conn.close()

In [12]:
def get_data_by_condition(table, fields, condition):
    conn = sqlite3.connect('data/futures.db')
    fields_str = ', '.join(fields)  # Convert the list of fields into a comma-separated string
    sql = f"SELECT {fields_str} FROM {table} WHERE {condition}"
    df = pd.read_sql_query(sql, conn)
    conn.close()
    return df

In [13]:
df = get_data_by_condition('shfe', ['symbol', 'date', 'close', 'volume', 'settle', 'variety'], "variety='RB'")
df

,symbol,date,close,volume,settle,variety
0,RB1101,20110104,4939.0,1924,4933.0,RB
1,RB1102,20110104,4864.0,112,4866.0,RB
2,RB1103,20110104,4819.0,208,4820.0,RB
3,RB1104,20110104,4821.0,238,4818.0,RB
4,RB1105,20110104,4822.0,275428,4823.0,RB
...,...,...,...,...,...,...
37648,RB2407,20231130,3954.0,12755,3946.0,RB
37649,RB2408,20231130,3951.0,997,3947.0,RB
37650,RB2409,20231130,3942.0,2286,3926.0,RB
37651,RB2410,20231130,3910.0,8761,3897.0,RB


In [37]:
conn = sqlite3.connect('futures.db')
df = pd.read_sql_query("SELECT * from shfe WHERE variety='RB'", conn)
conn.close
df

,symbol,date,open,high,low,close,volume,open_interest,turnover,settle,pre_settle,variety
0,RB1101,20110104,4929.0,4939.0,4920.0,4939.0,1924,4380.0,9.491092e+06,4933.0,4901.0,RB
1,RB1102,20110104,4864.0,4892.0,4864.0,4864.0,112,1262.0,5.449920e+05,4866.0,4863.0,RB
2,RB1103,20110104,4830.0,4830.0,4818.0,4819.0,208,3732.0,1.002560e+06,4820.0,4821.0,RB
3,RB1104,20110104,4800.0,4828.0,4800.0,4821.0,238,6578.0,1.146684e+06,4818.0,4804.0,RB
4,RB1105,20110104,4830.0,4843.0,4805.0,4822.0,275428,576934.0,1.328389e+09,4823.0,4803.0,RB
...,...,...,...,...,...,...,...,...,...,...,...,...
37648,RB2407,20231130,3957.0,3968.0,3911.0,3954.0,12755,107662.0,5.033123e+07,3946.0,3958.0,RB
37649,RB2408,20231130,3941.0,3960.0,3904.0,3951.0,997,24093.0,3.935159e+06,3947.0,3949.0,RB
37650,RB2409,20231130,3935.0,3949.0,3891.0,3942.0,2286,17376.0,8.974836e+06,3926.0,3935.0,RB
37651,RB2410,20231130,3915.0,3928.0,3873.0,3910.0,8761,40935.0,3.414162e+07,3897.0,3917.0,RB


In [28]:
max_row = df_symbol_contract_date['volume'].idxmax()
df_symbol_contract_date.loc[max_row]['symbol']

'RB1105'

In [ ]:
symbol_chain = commodity.SymbolChain('Steel', '黑色金属', json_file)
symbol_chain.add_symbol(symbol)
symbol_chain.add_symbol(symbol_i)
symbol_chain.add_symbol(symbol_j)
df_profit_c = symbol.get_profits(symbol_chain)
# df_profit_c.dropna(axis=0, how='all', subset=['现货利润', '盘面利润'], inplace=True)
symbol.calculate_data_rank(trace_back_months=60)

### 更新数据

- 通过AKshare接口更新现货价格数据
- 手动更新Choice导出数据（使用Microsoft Excel打开，自动更新后保存）

In [5]:
# 更新现货价格数据 - AKShare
# 追加最新数据
df_basis = symbol.update_akshare_file(mode='append')
# 2011、2012年数据待更新
# symbol_data.update_akshare_file(mode='period', start_date='20120101', end_date='20121231')

d:\Python3\lib\site-packages\akshare\futures\futures_basis.py:101: UserWarning: 20231202非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")
d:\Python3\lib\site-packages\akshare\futures\futures_basis.py:101: UserWarning: 20231203非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")
d:\Python3\lib\site-packages\akshare\futures\futures_basis.py:101: UserWarning: 20231209非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")
d:\Python3\lib\site-packages\akshare\futures\futures_basis.py:101: UserWarning: 20231210非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")
d:\Python3\lib\site-packages\akshare\futures\futures_basis.py:101: UserWarning: 20231216非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")
d:\Python3\lib\site-packages\akshare\futures\futures_basis.py:101: UserWarning: 20231217非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")
d:\Python3\lib\site-packages\akshare\futures\futures_basis.py:101: UserWarning: 20231223非交易日
  warnings.warn(f"{date.strftime('%Y%m%d')}非交易日")

## 基差-库存/仓单-利润分析

### 历史走势分析

前置条件：
- 非季节性品种确认

分析内容：
- 现货价格/期货价格（收盘价）、基差的历史趋势
- 基差率历史趋势，基差率历史分位
- 库存、仓单、库存消费比，库存、仓单的历史分位
- 现货利润和盘面利润，及其历史分位
- 现货月区域标记
- 多维指标共振区域标记

扩展功能：
- 图表可配置化
- 小图：时点跨期套利分析
- 小图：时点期限结构分析

In [ ]:
symbol.get_spot_months()
symbol.spot_months

In [ ]:
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
                    specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]],
                   vertical_spacing=0.01, 
                   subplot_titles=('基差分析', '基差率', '库存/仓单历史分位', '现货利润/盘面利润'), 
                   row_width=[0.1, 0.1, 0.1, 0.7])

# 创建主图：期货价格、现货价格、基差
fig_future_price = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['主力合约收盘价'], name='期货价格', 
                              marker_color='rgb(84,134,240)')
fig_spot_price = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['现货价格'], name='现货价格', marker_color='rgb(105,206,159)')
fig_basis = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['基差'], stackgroup='one', name='基差', 
                       marker_color='rgb(239,181,59)', showlegend=False)
fig.add_trace(fig_basis, secondary_y=True)
fig.add_trace(fig_future_price, row = 1, col = 1)
fig.add_trace(fig_spot_price, row = 1, col = 1)

# 创建辅图-基差率，并根据基差率正负配色
sign_color_mapping = {0:'green', 1:'red'}
fig_basis_rate = go.Bar(x=symbol.symbol_data['日期'], y = symbol.symbol_data['基差率'], name='基差率',
                        marker=dict(color=symbol.basis_color['基差率颜色'], colorscale=list(sign_color_mapping.values()),
                                    showscale=False),
                        showlegend=False,
                        hovertemplate='%{y:.2%}')
fig.add_trace(fig_basis_rate, row = 2, col = 1)

# 创建辅图-库存/仓单
fig_receipt = go.Scatter(x=symbol.symbol_data['日期'], y=symbol.symbol_data['仓单'], name='仓单', marker_color='rgb(239,181,59)')
fig_storage = go.Bar(x=symbol.symbol_data['日期'], y=symbol.symbol_data['库存'], name='库存', marker_color='rgb(234,69,70)')
fig.add_trace(fig_receipt, row = 3, col = 1, secondary_y=True)
fig.add_trace(fig_storage, row = 3, col = 1)

# 创建辅图-库存/仓单历史时间百分位，并根据分位配色
histroy_color_mapping ={1:'red', 2:'lightblue', 3:'lightblue', 4:'lightblue', 5:'green'}
# df_rank['仓单分位颜色'] = df_rank['仓单历史时间分位'].map(histroy_color_mapping)
# fig_receipt_rank = go.Scatter(x=df_rank['日期'], y=df_rank['仓单历史时间百分位'], name='仓单分位', marker_color='rgb(239,181,59)')
fig_receipt_rank = go.Scatter(x=symbol.data_rank['日期'], y=symbol.data_rank['仓单历史时间百分位'], name='仓单分位', mode='markers',
                              marker=dict(size=2, color=symbol.data_rank['仓单历史时间分位'], colorscale=list(histroy_color_mapping.values())),
                              showlegend=False,
                              hovertemplate='%{y:.2%}')
# fig.add_trace(fig_receipt_rank, row = 3, col = 1, secondary_y=True)
symbol.data_rank['库存分位颜色'] = symbol.data_rank['库存历史时间分位'].map(histroy_color_mapping)
# fig_storage_rank = go.Bar(x=df_rank['日期'], y=df_rank['库存历史时间百分位'], name='库存分位', marker_color='rgb(234,69,70)')
fig_storage_rank = go.Bar(x=symbol.data_rank['日期'], y=symbol.data_rank['库存历史时间百分位'], name='库存分位', marker_color=symbol.data_rank['库存分位颜色'],
                          hovertemplate='%{y:.2%}')
# fig.add_trace(fig_storage_rank, row = 3, col = 1)

# 创建辅图-现货利润/盘面利润
# fig_spot_profit = go.Scatter(x=df_profit['日期'], y=df_profit['现货利润'], name='现货利润', marker_color='rgb(239,181,59)')
# fig_future_profit = go.Bar(x=df_profit['日期'], y=df_profit['盘面利润'], name='盘面利润', marker_color='rgb(234,69,70)')
# fig.add_trace(fig_spot_profit, row = 4, col = 1, secondary_y=True)
# fig.add_trace(fig_future_profit, row = 4, col = 1)

# 创建辅图-现货利润/盘面利润历史时间分位
symbol.data_rank['盘面利润分位颜色'] = symbol.data_rank['盘面利润历史时间分位'].map(histroy_color_mapping)
fig_spot_profit = go.Scatter(x=symbol.data_rank['日期'], y=symbol.data_rank['现货利润历史时间百分位'], name='现货利润', mode='markers',
                             marker=dict(size=2, color=symbol.data_rank['现货利润历史时间分位'], colorscale=list(histroy_color_mapping.values())),
                             hovertemplate='%{y:.2%}')
fig.add_trace(fig_spot_profit, row = 4, col = 1, secondary_y=True)
fig_future_profit = go.Bar(x=symbol.data_rank['日期'], y=symbol.data_rank['盘面利润历史时间百分位'], name='盘面利润', marker_color=symbol.data_rank['盘面利润分位颜色'],
                           showlegend=False,
                           hovertemplate='%{y:.2%}')
fig.add_trace(fig_future_profit, row = 4, col = 1)



# 根据交易时间过滤空数据
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = [d.strftime("%Y-%m-%d") for d in trade_date]
dt_all = pd.date_range(start=symbol.symbol_data['日期'].iloc[0],end=symbol.symbol_data['日期'].iloc[-1])
dt_all = [d.strftime("%Y-%m-%d") for d in dt_all]
dt_breaks = list(set(dt_all) - set(trade_date))

for _, row in symbol.spot_months.iterrows():
    fig.add_shape(
        # 矩形
        type="rect",
        # 矩形的坐标
        x0=row['Start Date'],
        x1=row['End Date'],
        y0=0,
        y1=1,
        xref='x',
        yref='paper',
        # 矩形的颜色和透明度
        fillcolor="LightBlue",
        opacity=0.1,
        # 矩形的边框
        line_width=0,
        # 矩形在数据之下
        layer="below"
    )

select_synchronize_index_value = ['基差率', '库存|仓单', '现货利润|盘面利润']
# df_signals =symbol.get_signals(select_synchronize_index_value)
symbol.signals = pd.merge(symbol.symbol_data[['日期', '基差率']],
                        symbol.data_rank[['日期', '库存历史时间分位', '仓单历史时间分位', '现货利润历史时间分位', '盘面利润历史时间分位']],
                        on='日期', how='outer')
symbol.signals['基差率'] = symbol.signals['基差率'].map(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
# print(symbol.signals)
# # For other columns
# for col in ['库存历史时间分位', '仓单历史时间分位', '现货利润历史时间分位', '盘面利润历史时间分位']:
#     symbol.signals[col] = symbol.signals[col].map(lambda x: -1 if x == 5 else (0 if x != 1 else 1))
for col in ['库存历史时间分位', '仓单历史时间分位', '现货利润历史时间分位', '盘面利润历史时间分位']:
    symbol.signals[col] = symbol.signals[col].map(lambda x: -1 if x == 5 else (0 if x != 1 else 1)).fillna(0).astype(int)
print(symbol.signals)
symbol.signals['库存|仓单'] = symbol.signals['库存历史时间分位'] | symbol.signals['仓单历史时间分位']
symbol.signals['现货利润|盘面利润'] = symbol.signals['现货利润历史时间分位'] | symbol.signals['盘面利润历史时间分位']
if len(select_synchronize_index_value)!=0:
    symbol.signals['信号数量'] = symbol.signals[select_synchronize_index_value].sum(axis=1)

signal_nums = len(select_synchronize_index_value)
df_short_signals = symbol.signals[symbol.signals['信号数量']==-signal_nums]        
for _, row in df_short_signals.iterrows():
    next_day = row['日期'] + timedelta(days=1)
    fig.add_shape(
        type='circle',
        x0=row['日期'], x1=next_day,
        y0=1, y1=0.99,
        xref='x', yref='paper',
        fillcolor='green',
        line_color='green'
    )

# X轴坐标按照年-月显示
fig.update_xaxes(
    showgrid=True,
    zeroline=True,
    dtick="M1",  # 按月显示
    ticklabelmode="period",   # instant  period
    tickformat="%b\n%Y",
    rangebreaks=[dict(values=dt_breaks)],
    rangeslider_visible = False, # 下方滑动条缩放
    # 增加固定范围选择
    # rangeselector = dict(
    #     buttons = list([
    #         dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
    #         dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
    #         dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
    #         dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
    #         dict(step = 'all')
    #         ]))
)
#fig.update_traces(xbins_size="M1")
max_y = symbol.symbol_data['主力合约收盘价'] .max() * 1.05
min_y = symbol.symbol_data['主力合约收盘价'] .min() * 0.95
fig.update_layout(
    yaxis_range=[min_y,max_y],
    #autosize=False,
    #width=800,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    plot_bgcolor='WhiteSmoke',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis2_showgrid=False,
    hovermode='x unified',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)

fig.show()

In [7]:
self = symbol
df_rank = None
mode='time'
trace_back_months=3
quantiles=[0, 20, 40, 60, 80, 100]
ranks=[1, 2, 3, 4, 5]
data_list=['库存', '仓单', '现货利润', '盘面利润']

field = data_list[0]
df_append= pd.DataFrame()
df_append['日期'] = self.symbol_data['日期']
if trace_back_months == 'all':
    window_size = len(self.symbol_data)
else:
    window_size = trace_back_months * 20  # assuming 30 days per month
if mode=='time':
    value_field = field + '历史时间百分位'
    rank_field = field + '历史时间分位'
    df_append[value_field] = self.symbol_data[field].rolling(window=window_size, min_periods=1).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    quantiles = np.percentile(df_append[value_field].dropna(), quantiles)
elif mode=='value':
    value_field = field + '历史数值百分位'
    rank_field = field + '历史数值分位'
    df_append[value_field] = self.symbol_data[field].rolling(window=window_size, min_periods=1).apply(lambda x: (x[-1] - np.min(x)) / (np.max(x) - np.min(x)))
    quantiles = list(map(lambda x: x/100, quantiles))
else:
    None

df_append[rank_field] = pd.cut(df_append[value_field].dropna(), bins=quantiles, labels=ranks, include_lowest=True, duplicates='drop', right=False)
if df_rank==None:
    df_rank = df_append
else:
    df_rank = pd.merge(df_rank, df_append, on='日期', how='outer')

df_rank = pd.merge(df_rank, symbol.symbol_data[['日期', field]], on='日期', how='outer')


In [31]:
self = symbol
df_rank = pd.DataFrame()
mode='time'
trace_back_months=1
quantiles=[0, 20, 40, 60, 80, 100]
ranks=[1, 2, 3, 4, 5]
data_list=['库存', '仓单', '现货利润', '盘面利润']
field = data_list[0]

df_append= pd.DataFrame()
df_append['日期'] = self.symbol_data['日期']
if mode=='time':
    value_field = field + '历史时间百分位'
    rank_field = field + '历史时间分位'
elif mode=='value':
    value_field = field + '历史数值百分位'
    rank_field = field + '历史数值分位'
else:
    None

if trace_back_months == 'all':
    window_size = len(self.symbol_data)
else:
    # Assuming that data is daily, convert months to days
    window_size = trace_back_months * 20

if mode=='time':
    df_append[value_field] = self.symbol_data[field].expanding().apply(lambda x: (x.rank(method='min') / len(x)).iloc[-1])
    quantiles = np.percentile(df_append[value_field].dropna(), quantiles)
elif mode=='value':
    df_append[value_field] = self.symbol_data[field].expanding().apply(lambda x: (x.iloc[-1] - x.min()) / (x.max() - x.min()))
    quantiles = list(map(lambda x: x/100, quantiles))

df_append[rank_field] = pd.cut(df_append[value_field].dropna(), bins=quantiles, labels=ranks, include_lowest=True, duplicates='drop', right=False)
if df_rank.empty:
    df_rank = df_append
else:
    df_rank = pd.merge(df_rank, df_append, on='日期', how='outer')

df_rank = pd.merge(df_rank, symbol.symbol_data[['日期', field]], on='日期', how='outer')

### 历史水位分析

基差率-库存消费比-利润率

历史时间比例分位

In [ ]:
# df1 = symbol.history_time_ratio('库存', df_rank=merged_data)
# df2 = symbol.history_time_ratio('库存', df_rank=merged_data, mode='value')


In [ ]:
# 将累计销量数据转化为当月销量数据

# 假设您提供的数据保存在一个名为df的dataframe中，字段包含日期和累计销量
# 例如，df的前五行如下：
#          日期   累计销量
# 0 2020-01-31  1000
# 1 2020-02-29  1500
# 2 2020-03-31  1800
# 3 2020-04-30  2200
# 4 2020-05-31  2500

# 定义一个函数，计算当月销量值
def calc_monthly_sales(df):
    # 创建一个空的列表，用于存储当月销量值
    monthly_sales = []
    # 遍历dataframe的每一行，获取日期和累计销量
    for i, row in df.iterrows():
        # 获取日期
        date = row['日期']
        # 获取累计销量
        cum_sales = row['累计销量']
        # 如果是第一行，那么需要判断是否是1月份
        if i == 0:
            # 如果是1月份，那么当月销量值就等于累计销量
            if date.month == 1:
                monthly_sales.append(cum_sales)
            # 如果不是1月份，那么当月销量值就设为NaN
            else:
                monthly_sales.append(np.nan)
        # 如果不是第一行，那么需要判断当前月份与上一行的月份是否相邻
        else:
            # 获取上一行的日期
            prev_date = df.loc[i-1, '日期']
            # 如果当前月份与上一行的月份相邻，那么当月销量值就等于累计销量减去上一行的累计销量
            if date.month == prev_date.month + 1 or (date.month == 1 and prev_date.month == 12):
                monthly_sales.append(cum_sales - df.loc[i-1, '累计销量'])
            # 如果当前月份与上一行的月份不相邻，那么当月销量值就设为NaN
            else:
                monthly_sales.append(np.nan)
    # 返回列表
    return monthly_sales

# 调用函数，得到一个列表，存储当月销量值
monthly_sales = calc_monthly_sales(df)

# 在原始的dataframe中，创建一个新的列，存储当月销量值
df['当月销量'] = monthly_sales

# 打印dataframe的前五行，查看结果
print(df.head())

#          日期   累计销量  当月销量
# 0 2020-01-31  1000   1000.0
# 1 2020-02-29  1500    500.0
# 2 2020-03-31  1800    300.0
# 3 2020-04-30  2200    400.0
# 4 2020-05-31  2500    300.0


## 季节性分析

### 基差率季节分析

In [ ]:
df_rb0['年度'] = df_rb0['日期'].dt.year
df_rb0['年内日期'] = df_rb0['日期'].dt.strftime('1900-%m-%d')
fig_basis_rate_season = px.line(df_rb0,
                                x='年内日期',
                                y='基差率',
                                color='年度',
                                #color_discrete_sequence=px.colors.qualitative.G10)
                                color_discrete_sequence=['lightgray', 'lightblue', 'orange', 'red'])
fig_basis_rate_season.update_layout(
    title={
        'text':'基差率季节分析',
        'xanchor':'center'},
    margin=dict(l=10, r=10, t=40, b=10)
)

fig_basis_rate_season.show()

### 基差率月度涨跌统计

### 基差率频率分布

### 库存季节性分析

## 跨期分析

### 期限结构

In [ ]:
# 加载合约基础数据
futures_comm_info = pd.read_excel('data/common_info.xlsx')
spec_contact_list = futures_comm_info[futures_comm_info.合约名称.str.startswith('螺纹钢')]
fig_term = make_subplots(specs=[[{"secondary_y": True}]])
fig_term.add_trace(go.Scatter(x=spec_contact_list['合约代码'], y=spec_contact_list['现价']))
# 获取最新现货价格
spot_price = df_rb0[df_rb0['现货']!=0]['现货'].iloc[-1]
fig_term.add_hline(y=spot_price)
fig_term.update_layout(
    title={
        'text':'期限结构'
    },
    #autosize=False,
    width=800,
    #height=800,
    margin=dict(l=10, r=10, t=40, b=10)
)
fig_term.show()

### 套利分析

#### 价差分析-多期排列

#### 价差分析-跨期价差矩阵

#### 基差-月差分析

#### 价差季节性分析

## 库存

### 库存周期

#### 期转现

#### 交割统计

## 利润

### 现货利润

### 期货盘面利润

### 利润期限结构

## 综合分析

### 基差-库存-利润分析

### 基差-月差分析

### 期限结构-库存/仓单分析